In [1]:
import pandas as pd, numpy as np, datetime, math, matplotlib.pyplot as plt, calendar
%matplotlib inline

/Users/terencekwt/Code/datadive_201608_ci/env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
marin = pd.read_csv('../data/StandardizedData/Marin_Merged.csv')
team = pd.read_csv('../data/StandardizedData/Terrestrial_Vertebrate_Cols_Edited.csv')

/Users/terencekwt/Code/datadive_201608_ci/env/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,20,21,22,27,28,30,31,32,33,35,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#columns of the marin dataset
marin.columns

Index([u'Unnamed: 0', u'Project.ID', u'Deployment.ID', u'Image.ID',
       u'Location', u'Photo.Type', u'Photo.Type.Identified.by',
       u'Genus.Species', u'Uncertainty', u'IUCN.Identification.Number',
       u'Date_Time.Captured', u'Age', u'Sex', u'Individual.ID', u'Count',
       u'Animal.recognizable..Y.N.', u'individual.Animal.notes',
       u'Project.ID.x', u'Event', u'Array.Name', u'Deployment.Location.ID',
       u'Longitude.Resolution', u'Latitude.Resolution',
       u'Camera.Deployment.Begin.Date', u'Camera.Deployment.End.Date',
       u'Bait.Type', u'Bait.Description', u'Feature.Type',
       u'Feature.Type.methodology', u'Camera.ID', u'Quiet.Period.Setting',
       u'Restriction.on.access', u'Camera.Failure.Details', u'Project.ID.y',
       u'Make', u'Model', u'Serial.Number', u'Year.Purchased', u'Genus',
       u'Species'],
      dtype='object')

In [4]:
# Number of locations 

len(set(marin['Deployment.Location.ID']))

8

In [5]:
# Number of projects

len(set(marin['Project.ID']))

1

In [6]:
# SELECT the projections that we actually need
marin_simple = marin[['Project.ID',
'Deployment.Location.ID',
'Camera.Deployment.Begin.Date',
'Camera.Deployment.End.Date']]


marin_simple_per_location = marin_simple.groupby(['Project.ID', 'Deployment.Location.ID', 'Camera.Deployment.Begin.Date', 'Camera.Deployment.End.Date'])
marin_simple_per_location

In [7]:
marin_simple_per_location = marin_simple_per_location.count().reset_index()

In [8]:
marin_simple_per_location['Camera.Deployment.Begin.Date_datetime'] = marin_simple_per_location['Camera.Deployment.Begin.Date'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%Y-%m-%d'))

marin_simple_per_location['Camera.Deployment.End.Date_datetime'] = marin_simple_per_location['Camera.Deployment.End.Date'].apply(lambda x: 
                                                                    datetime.datetime.strptime(x, '%Y-%m-%d'))

marin_simple_per_location['trapnights'] = (marin_simple_per_location['Camera.Deployment.End.Date_datetime'] 
                                          - 
        marin_simple_per_location['Camera.Deployment.Begin.Date_datetime']).astype(pd.Timedelta).apply(lambda l: l.days)

In [9]:
marin_simple_per_location.head()

,Project.ID,Deployment.Location.ID,Camera.Deployment.Begin.Date,Camera.Deployment.End.Date,Camera.Deployment.Begin.Date_datetime,Camera.Deployment.End.Date_datetime,trapnights
0,ChedaJewel,WW63,2014-09-22,2014-10-17,2014-09-22,2014-10-17,25
1,ChedaJewel,WW63,2014-10-17,2014-12-17,2014-10-17,2014-12-17,61
2,ChedaJewel,WW63,2014-12-17,2015-02-04,2014-12-17,2015-02-04,49
3,ChedaJewel,WW63,2015-02-04,2015-04-07,2015-02-04,2015-04-07,62
4,ChedaJewel,WW63,2015-04-07,2015-06-02,2015-04-07,2015-06-02,56


In [43]:
#Generate a trap night count for each unique {ProjectId + DeploymentId + Month}

from dateutil.relativedelta import relativedelta

def getDaysInMonth(date):
    '''
    For a specific day d, it will return the number of days between d and the end of the month
    '''
    endDate = date + relativedelta(day=1, months=+1, days=-1)
    return (endDate - date).days + 1

def genCountByMonth(start, end):
    '''
    Given a date range, it will return a dictionary of tuples for (month, trapnight counts)
    '''
    totalcount = (end - start).days
    results = {}
    while totalcount > 0:
        daysInCurrentMonth = min(totalcount,getDaysInMonth(start))
        if(start.month in results.keys()):
            results[start.month] += daysInCurrentMonth
        else:
            results[start.month] = daysInCurrentMonth
        totalcount -= daysInCurrentMonth
        start += relativedelta(day=1, months=+1)
    return results

print genCountByMonth(marin_simple_per_location['Camera.Deployment.Begin.Date_datetime'][0], marin_simple_per_location['Camera.Deployment.End.Date_datetime'][0])

{9: 9, 10: 16}


In [44]:
for i in range(0,len(marin_simple_per_location)):
    print genCountByMonth(marin_simple_per_location['Camera.Deployment.Begin.Date_datetime'][i], marin_simple_per_location['Camera.Deployment.End.Date_datetime'][i])

{9: 9, 10: 16}
{10: 15, 11: 30, 12: 16}
{1: 31, 2: 3, 12: 15}
{2: 25, 3: 31, 4: 6}
{4: 24, 5: 31, 6: 1}
{6: 29, 7: 21}
{8: 31, 9: 13, 7: 10}
{9: 17, 10: 31, 11: 18}
{1: 12, 11: 12, 12: 31}
{1: 19, 2: 21}
{2: 8, 3: 28}
{9: 9, 10: 16}
{10: 15, 11: 30, 12: 16}
{1: 31, 2: 3, 12: 15}
{2: 25, 3: 31, 4: 6}
{4: 24, 5: 31, 6: 1}
{6: 29, 7: 14}
{8: 31, 9: 13, 7: 17}
{9: 17, 10: 31, 11: 18}
{1: 12, 11: 12, 12: 31}
{1: 19, 2: 21}
{9: 9, 10: 16}
{10: 15, 11: 30, 12: 16}
{1: 31, 2: 3, 12: 15}
{2: 25, 3: 31, 4: 6}
{4: 24, 5: 31, 6: 1}
{6: 29, 7: 14}
{8: 31, 9: 13, 7: 17}
{9: 17, 10: 31, 11: 18}
{1: 12, 11: 12, 12: 31}
{1: 19, 2: 21}
{2: 8, 3: 28}
{9: 9, 10: 26}
{10: 5, 11: 30, 12: 21}
{1: 31, 2: 2, 12: 10}
{2: 26, 3: 31, 4: 13}
{6: 16, 7: 21}
{8: 31, 9: 29, 7: 10}
{9: 1, 10: 31, 11: 24}
{1: 14, 11: 6, 12: 31}
{1: 17, 2: 21}
{2: 8, 3: 31, 4: 7}
{9: 9, 10: 23}
{10: 8, 11: 30, 12: 16}
{1: 31, 2: 3, 12: 15}
{2: 25, 3: 25}
{3: 6, 4: 13}
{4: 17, 5: 31, 6: 1}
{8: 31, 9: 13, 7: 17}
{9: 17, 10: 31, 11: 24}
{1